## Video MCTF coding (Hierarchical B-frames)

In [ ]:
from IPython.display import Video

In [ ]:
#!pip install -r ../requirements.txt

### Help about basic functionality

In [ ]:
!python3 ../src/MCTF.py -h

### Help to encode

In [ ]:
!python3 ../src/MCTF.py encode -h

### Encode and decoding a remote video

In [ ]:
Video("http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4")

In [ ]:
# All files will be saved to /tmp automatically
# Use -o for original video, --num_gops to specify how many GOPs to encode
# Total frames = gop_size * num_gops (16 * 2 = 32 frames)
!python3 ../src/MCTF.py encode -o http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4 --gop_size 16 --num_gops 2

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
# Video is automatically created at /tmp/decoded.mp4
Video("/tmp/decoded.mp4")

### Encode and decode a local video

In [ ]:
!wget http://www.hpca.ual.es/~vruiz/videos/coastguard_352x288x30x420x300.avi

In [ ]:
# Encode with -o (original) and specify number of GOPs
# Total frames = gop_size * num_gops (16 * 10 = 160 frames)
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --gop_size 16 --num_gops 10

In [ ]:
# Decodes all frames and creates MP4 automatically
!python3 ../src/MCTF.py decode

In [ ]:
Video("/tmp/decoded.mp4")

### Default encoding and decoding

In [ ]:
!rm /tmp/encoded* /tmp/decoded* 2>/dev/null; echo "Cleaned /tmp files"

In [ ]:
!python3 ../src/MCTF.py encode

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
Video("/tmp/decoded.mp4")

### Using 2D-DWT

In [ ]:
# -T is from VCF (transform), -o is for original video
!python ../src/MCTF.py encode -T 2D-DWT -o coastguard_352x288x30x420x300.avi --gop_size 16 --num_gops 5

In [ ]:
!python3 ../src/MCTF.py decode -T 2D-DWT

In [ ]:
Video("/tmp/decoded.mp4")

### Using hierarchical B-frame structure

In [ ]:
!python ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --hierarchical --gop_size 16 --num_gops 5

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
Video("/tmp/decoded.mp4")

### Using fast motion estimation

In [ ]:
!python ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --fast --gop_size 16 --num_gops 5

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
Video("/tmp/decoded.mp4")

### Different quantization steps

In [ ]:
# Lower QSS = better quality (-q is from VCF)
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi -q 16 --fast --gop_size 16 --num_gops 5

In [ ]:
!python3 ../src/MCTF.py decode -q 16

In [ ]:
Video("/tmp/decoded.mp4")

In [ ]:
# Higher QSS = more compression (-q is from VCF)
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi -q 64 --fast --gop_size 16 --num_gops 5

In [ ]:
!python3 ../src/MCTF.py decode -q 64

In [ ]:
Video("/tmp/decoded.mp4")

### Hierarchical vs Non-Hierarchical B-frames Comparison

**Key Difference:**
- **Without `--hierarchical`**: All B-frames reference the same two anchors (I and P). Simple but less efficient.
- **With `--hierarchical`**: B-frames reference closer decoded frames. Better prediction = lower residuals = better compression.

**Example GOP=8:**
```
Simple:      I  B  B  B  B  B  B  P
             All B refs → (I, P)

Hierarchical: I  B  B  B  P  B  B  B
              Mid B refs → (I, P)
              Left B refs → (I, Mid)
              Right B refs → (Mid, P)
```

Hierarchical gives better compression because closer references = smaller motion vectors and residuals.

In [ ]:
# Clean up
!rm /tmp/encoded* /tmp/decoded* 2>/dev/null; echo "Cleaned /tmp files"

In [ ]:
# WITHOUT hierarchical - simple B-frame structure
# All B-frames reference same two anchors (I and P)
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --gop_size 8 --num_gops 3 --fast -O /tmp/simple

In [ ]:
!python3 ../src/MCTF.py decode -i /tmp/simple -O /tmp/simple_decoded

In [ ]:
Video("/tmp/simple_decoded.mp4")

In [ ]:
# WITH hierarchical - B-frames reference closer decoded frames
# Better prediction = better compression
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --gop_size 8 --num_gops 3 --fast --hierarchical -O /tmp/hierarchical

In [ ]:
!python3 ../src/MCTF.py decode -i /tmp/hierarchical -O /tmp/hierarchical_decoded

In [ ]:
Video("/tmp/hierarchical_decoded.mp4")

### BPP Comparison

In [ ]:
import os
import glob

# Calculate total size of encoded files
def calculate_encoded_size(prefix):
    total_size = 0
    # Get all encoded files (frames + motion vectors + metadata)
    patterns = [
        f"{prefix}_*.png",
        f"{prefix}_*_mv.npz",
        f"{prefix}_*_type.txt"
    ]
    for pattern in patterns:
        files = glob.glob(pattern)
        for f in files:
            if os.path.exists(f):
                total_size += os.path.getsize(f)
    return total_size

# Video parameters
width = 352
height = 288
num_frames = 24  # 3 GOPs of 8 frames each
total_pixels = width * height * num_frames

# Calculate sizes
simple_size = calculate_encoded_size("/tmp/simple")
hierarchical_size = calculate_encoded_size("/tmp/hierarchical")

# Calculate BPP
simple_bpp = (simple_size * 8) / total_pixels if total_pixels > 0 else 0
hierarchical_bpp = (hierarchical_size * 8) / total_pixels if total_pixels > 0 else 0

# Display comparison
print("="*60)
print("COMPRESSION COMPARISON: Simple vs Hierarchical B-frames")
print("="*60)
print(f"Video: {width}x{height}, {num_frames} frames")
print(f"Total pixels: {total_pixels:,}")
print()
print("Simple B-frame structure (all B refs → I,P):")
print(f"  Total size: {simple_size:,} bytes ({simple_size/1024:.2f} KB)")
print(f"  BPP: {simple_bpp:.6f} bits/pixel")
print()
print("Hierarchical B-frame structure (B refs → closer frames):")
print(f"  Total size: {hierarchical_size:,} bytes ({hierarchical_size/1024:.2f} KB)")
print(f"  BPP: {hierarchical_bpp:.6f} bits/pixel")
print()
if simple_size > 0:
    improvement = ((simple_size - hierarchical_size) / simple_size) * 100
    print(f"Improvement: {improvement:.2f}% smaller with hierarchical")
    print(f"Compression ratio: {simple_size/hierarchical_size:.2f}x")
print("="*60)

**Why is hierarchical better?**

Hierarchical B-frames achieve better compression because:
1. **Closer references**: B-frames reference temporally closer frames, resulting in smaller motion vectors
2. **Lower residuals**: Better prediction from closer references means smaller residual errors
3. **Better rate-distortion**: Lower BPP with similar or better quality

This is the same principle used in modern video codecs like H.264, H.265, and AV1!